This page aims to replicate most of the methods discussed on [Mitchell A.Pertersen's standard errors programming advice page](http://www.kellogg.northwestern.edu/faculty/petersen/htm/papers/se/se_programming.htm).  

In order to have a basis for comparison,we'll use Petersen's [sample dataset](http://www.kellogg.northwestern.edu/faculty/petersen/htm/papers/se/test_data.txt) and compare our results with those [reported on his page](http://www.kellogg.northwestern.edu/faculty/petersen/htm/papers/se/test_data.htm).


Sources:  
http://www.vincentgregoire.com/standard-errors-in-python/

http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.get_robustcov_results.html

Petersen, Mitchell A. “Estimating Standard Errors in Finance Panel Data Sets: Comparing Approaches.” The Review of Financial Studies 22, no. 1 (January 1, 2009): 435–80. https://doi.org/10.1093/rfs/hhn053.


In [1]:
import pandas as pd
import statsmodels.stats.sandwich_covariance as sw
import statsmodels.formula.api as sm
import numpy as np
from urllib import urlopen
url=r'http://www.kellogg.northwestern.edu/faculty/petersen/htm/papers/se/test_data.txt'
df=pd.read_table(urlopen(url),names=['firmid','year','x','y'],
                delim_whitespace=True)
df.head()

,firmid,year,x,y
0,1,1,-1.113973,2.251535
1,1,2,-0.080854,1.242346
2,1,3,-0.237607,-1.426376
3,1,4,-0.152486,-1.109394
4,1,5,-0.001426,0.914686


# OLS and statsmodels
## OLS Coefficients and standard errors
`use_t` parameter tells statsmodels to use t_statistics to compute the p-values.

In [2]:
ols=sm.ols(formula='y ~ x',data=df).fit(use_t=True)
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     1311.
Date:                Sun, 22 Oct 2017   Prob (F-statistic):          4.25e-255
Time:                        16:16:04   Log-Likelihood:                -10573.
No. Observations:                5000   AIC:                         2.115e+04
Df Residuals:                    4998   BIC:                         2.116e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0297      0.028      1.047      0.295        -0.026     0.085
x              1.0348      0.029     36.204      0.000         0.979     1.091
==============================================================================
Omnibus:                        4.912   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.862
Skew:                           0.070   Prob(JB):                       0.0880
Kurtosis:                       3.063   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## OLS coefficients and White Standard Errors
Adding heteroscedasticity-consistent standard errors is not much harder.The `cov_type` parameter can take many values,for heteroscedasticity-consistent standard errors different implementations take the value `HC0` (the original White estimator) to `HC3`.For more details about `HC0`,`HC`... refer to (Zeileis, Achim. “Econometric Computing with HC and HAC Covariance Matrix Estimators.” Paper, 2004. http://epub.wu.ac.at/520/.)

In [3]:
robust_ols=sm.ols(formula='y ~ x',data=df).fit(cov_type='HC1',
                    use_t=True)
robust_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     1328.
Date:                Sun, 22 Oct 2017   Prob (F-statistic):          4.29e-258
Time:                        16:26:28   Log-Likelihood:                -10573.
No. Observations:                5000   AIC:                         2.115e+04
Df Residuals:                    4998   BIC:                         2.116e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0297      0.028      1.047      0.295        -0.026     0.085
x              1.0348      0.028     36.444      0.000         0.979     1.091
==============================================================================
Omnibus:                        4.912   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.862
Skew:                           0.070   Prob(JB):                       0.0880
Kurtosis:                       3.063   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

## OLS coefficients and standard errors clustered by Firm or Year
Clustering can also be acheived by passing `cluster` to the `cov_type` parameter.You also need to give an additional parameter `cov_kwds`,which indicates which group to clusteron.The parameters takes an arrays of labels,which can be the columns of a pandas DataFrame as in this example.

### cluster on firmid

In [4]:
cluster_firm_ols=sm.ols(formula='y ~ x',data=df).fit(
    cov_type='cluster',cov_kwds={'groups':df['firmid']},
    use_t=True)
cluster_firm_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     418.3
Date:                Sun, 22 Oct 2017   Prob (F-statistic):           5.61e-68
Time:                        16:30:14   Log-Likelihood:                -10573.
No. Observations:                5000   AIC:                         2.115e+04
Df Residuals:                    4998   BIC:                         2.116e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0297      0.067      0.443      0.658        -0.102     0.161
x              1.0348      0.051     20.453      0.000         0.935     1.134
==============================================================================
Omnibus:                        4.912   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.862
Skew:                           0.070   Prob(JB):                       0.0880
Kurtosis:                       3.063   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### cluster on year

In [5]:
cluster_firm_ols=sm.ols(formula='y ~ x',data=df).fit(
    cov_type='cluster',cov_kwds={'groups':df['year']},
    use_t=True)
cluster_firm_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     960.6
Date:                Sun, 22 Oct 2017   Prob (F-statistic):           1.86e-10
Time:                        16:31:51   Log-Likelihood:                -10573.
No. Observations:                5000   AIC:                         2.115e+04
Df Residuals:                    4998   BIC:                         2.116e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0297      0.023      1.269      0.236        -0.023     0.083
x              1.0348      0.033     30.993      0.000         0.959     1.110
==============================================================================
Omnibus:                        4.912   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.862
Skew:                           0.070   Prob(JB):                       0.0880
Kurtosis:                       3.063   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### OLS coefficients and standard errors clustered by firm and year
Clustering along two dimensions is as easy as clutering along one dimension,all you have to do is pass an array of two columns as the group.The only caveat is that the group cannot be a `pandas.DataFrame` (while it can be a `Series`),you need to encapsulate it in `numpy.array()`

In [8]:
cluster_2ways_ols=sm.ols(formula='y ~ x',data=df).fit(
    cov_type='cluster',cov_kwds={'groups':np.array(df[['firmid','year']])},
    use_t=True)
cluster_2ways_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     373.3
Date:                Sun, 22 Oct 2017   Prob (F-statistic):           1.23e-08
Time:                        16:36:45   Log-Likelihood:                -10573.
No. Observations:                5000   AIC:                         2.115e+04
Df Residuals:                    4998   BIC:                         2.116e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0297      0.065      0.456      0.659        -0.118     0.177
x              1.0348      0.054     19.322      0.000         0.914     1.156
==============================================================================
Omnibus:                        4.912   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.862
Skew:                           0.070   Prob(JB):                       0.0880
Kurtosis:                       3.063   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### OLS standard errors clustering with N dimensions
There is no built-in function for $N-way$ clustering in python when $N>2$,one workaround is to use the `rpy2` package to link Python with `R` and estimate the standard errors with the `multiwayvcov` library.Refer to http://www.vincentgregoire.com/standard-errors-in-python/


## OLS coefficients and standard errors with firm and/or year fixed effects
One way to add fixed effects is by adding the corresponding dummy variables.Thankfully this is quite easy to do within a formula by using `C(var)` where `var` is the label variable.

In [12]:
firm_fe_ols=sm.ols(formula='y ~ x + C(firmid)',data=df).fit(
    use_t=True)
firm_fe_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     16.69
Date:                Sun, 22 Oct 2017   Prob (F-statistic):               0.00
Time:                        17:00:55   Log-Likelihood:                -8532.8
No. Observations:                5000   AIC:                         1.807e+04
Df Residuals:                    4499   BIC:                         2.133e+04
Df Model:                         500                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept            1.0518      0.445      2.366      0.018         0.180     1.923
C(firmid)[T.2]      -2.9039      0.629     -4.619      0.000        -4.136    -1.672
C(firmid)[T.3]       0.0446      0.629      0.071      0.943        -1.187     1.277
C(firmid)[T.4]      -4.2858      0.629     -6.817      0.000        -5.518    -3.054
C(firmid)[T.5]      -0.8661      0.631     -1.372      0.170        -2.103     0.371
C(firmid)[T.6]       0.4613      0.629      0.734      0.463        -0.771     1.693
C(firmid)[T.7]      -0.4831      0.630     -0.767      0.443        -1.718     0.752
C(firmid)[T.8]      -2.5868      0.629     -4.115      0.000        -3.819    -1.355
C(firmid)[T.9]      -0.4951      0.629     -0.788      0.431        -1.727     0.737
C(firmid)[T.10]     -0.9544      0.631     -1.512      0.131        -2.192     0.283
C(firmid)[T.11]     -3.0389      0.630     -4.823      0.000        -4.274    -1.804
C(firmid)[T.12]     -0.1493      0.629     -0.237      0.812        -1.382     1.083
C(firmid)[T.13]     -3.5822      0.629     -5.698      0.000        -4.814    -2.350
C(firmid)[T.14]      0.0058      0.629      0.009      0.993        -1.227     1.238
C(firmid)[T.15]     -1.7824      0.632     -2.821      0.005        -3.021    -0.544
C(firmid)[T.16]      0.2572      0.629      0.409      0.683        -0.976     1.490
C(firmid)[T.17]     -2.4279      0.629     -3.857      0.000        -3.662    -1.194
C(firmid)[T.18]     -0.6286      0.629     -0.999      0.318        -1.862     0.605
C(firmid)[T.19]     -1.7537      0.629     -2.789      0.005        -2.986    -0.521
C(firmid)[T.20]     -1.5594      0.630     -2.476      0.013        -2.794    -0.325
C(firmid)[T.21]      0.2231      0.629      0.355      0.723        -1.009     1.455
C(firmid)[T.22]      0.0067      0.629      0.011      0.991        -1.226     1.239
C(firmid)[T.23]     -0.7357      0.629     -1.170      0.242        -1.968     0.496
C(firmid)[T.24]     -1.6778      0.629     -2.666      0.008        -2.911    -0.444
C(firmid)[T.25]     -0.2716      0.631     -0.431      0.667        -1.508     0.965
C(firmid)[T.26]     -0.9912      0.629     -1.577      0.115        -2.223     0.241
C(firmid)[T.27]     -3.7442      0.629     -5.957      0.000        -4.976    -2.512
C(firmid)[T.28]     -0.8990      0.630     -1.427      0.154        -2.134     0.336
C(firmid)[T.29]     -4.7759      0.629     -7.595      0.000        -6.008    -3.543
C(firmid)[T.30]     -1.9464      0.629     -3.092      0.002        -3.180    -0.713
C(firmid)[T.31]     -0.8225      0.629     -1.307      0.191        -2.056     0.411
C(firmid)[T.32]      2.5302      0.629      4.025      0.000         1.298     3.762
C(firmid)[T.33]     -0.5626      0.630     -0.893      0.372        -1.798     0.673
C(firmid)[T.34]     -2.

In [13]:
firm_fe_ols=sm.ols(formula='y ~ x + C(year)',data=df).fit(
    use_t=True)
firm_fe_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     131.6
Date:                Sun, 22 Oct 2017   Prob (F-statistic):          7.13e-245
Time:                        17:01:58   Log-Likelihood:                -10570.
No. Observations:                5000   AIC:                         2.116e+04
Df Residuals:                    4989   BIC:                         2.123e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept         0.1411      0.090      1.573      0.116        -0.035     0.317
C(year)[T.2]     -0.0119      0.127     -0.094      0.925        -0.261     0.237
C(year)[T.3]     -0.1453      0.127     -1.145      0.252        -0.394     0.103
C(year)[T.4]     -0.2038      0.127     -1.607      0.108        -0.452     0.045
C(year)[T.5]     -0.0604      0.127     -0.476      0.634        -0.309     0.188
C(year)[T.6]     -0.1312      0.127     -1.034      0.301        -0.380     0.117
C(year)[T.7]     -0.1975      0.127     -1.557      0.120        -0.446     0.051
C(year)[T.8]     -0.1555      0.127     -1.225      0.220        -0.404     0.093
C(year)[T.9]     -0.1535      0.127     -1.210      0.226        -0.402     0.095
C(year)[T.10]    -0.0556      0.127     -0.438      0.661        -0.304     0.193
x                 1.0351      0.029     36.160      0.000         0.979     1.091
==============================================================================
Omnibus:                        4.804   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.091   Jarque-Bera (JB):                4.752
Skew:                           0.069   Prob(JB):                       0.0929
Kurtosis:                       3.061   Cond. No.                         10.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
firm_year_fe_ols = sm.ols(formula='y ~ x + C(year) + C(firmid)', data=df).fit(use_t=True)
firm_year_fe_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.651
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     16.43
Date:                Sun, 22 Oct 2017   Prob (F-statistic):               0.00
Time:                        17:03:20   Log-Likelihood:                -8525.9
No. Observations:                5000   AIC:                         1.807e+04
Df Residuals:                    4490   BIC:                         2.140e+04
Df Model:                         509                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept            1.1656      0.448      2.599      0.009         0.287     2.044
C(year)[T.2]        -0.0202      0.089     -0.227      0.820        -0.195     0.154
C(year)[T.3]        -0.1440      0.089     -1.621      0.105        -0.318     0.030
C(year)[T.4]        -0.2070      0.089     -2.329      0.020        -0.381    -0.033
C(year)[T.5]        -0.0651      0.089     -0.733      0.464        -0.239     0.109
C(year)[T.6]        -0.1295      0.089     -1.458      0.145        -0.304     0.045
C(year)[T.7]        -0.2027      0.089     -2.280      0.023        -0.377    -0.028
C(year)[T.8]        -0.1591      0.089     -1.790      0.073        -0.333     0.015
C(year)[T.9]        -0.1549      0.089     -1.743      0.081        -0.329     0.019
C(year)[T.10]       -0.0548      0.089     -0.616      0.538        -0.229     0.119
C(firmid)[T.2]      -2.9040      0.628     -4.621      0.000        -4.136    -1.672
C(firmid)[T.3]       0.0446      0.628      0.071      0.943        -1.187     1.276
C(firmid)[T.4]      -4.2857      0.628     -6.820      0.000        -5.517    -3.054
C(firmid)[T.5]      -0.8664      0.631     -1.373      0.170        -2.103     0.371
C(firmid)[T.6]       0.4613      0.628      0.734      0.463        -0.770     1.693
C(firmid)[T.7]      -0.4834      0.630     -0.767      0.443        -1.718     0.751
C(firmid)[T.8]      -2.5869      0.628     -4.117      0.000        -3.818    -1.355
C(firmid)[T.9]      -0.4951      0.628     -0.788      0.431        -1.727     0.736
C(firmid)[T.10]     -0.9548      0.631     -1.513      0.130        -2.192     0.282
C(firmid)[T.11]     -3.0391      0.630     -4.825      0.000        -4.274    -1.805
C(firmid)[T.12]     -0.1494      0.629     -0.238      0.812        -1.381     1.082
C(firmid)[T.13]     -3.5822      0.628     -5.701      0.000        -4.814    -2.351
C(firmid)[T.14]      0.0057      0.629      0.009      0.993        -1.226     1.238
C(firmid)[T.15]     -1.7828      0.632     -2.822      0.005        -3.021    -0.545
C(firmid)[T.16]      0.2573      0.629      0.409      0.682        -0.975     1.490
C(firmid)[T.17]     -2.4281      0.629     -3.859      0.000        -3.661    -1.195
C(firmid)[T.18]     -0.6288      0.629     -0.999      0.318        -1.862     0.604
C(firmid)[T.19]     -1.7537      0.628     -2.790      0.005        -2.986    -0.522
C(firmid)[T.20]     -1.5596      0.629     -2.478      0.013        -2.793    -0.326
C(firmid)[T.21]      0.2230      0.628      0.355      0.723        -1.009     1.455
C(firmid)[T.22]      0.0069      0.629      0.011      0.991        -1.225     1.239
C(firmid)[T.23]     -0.7357      0.628     -1.171      0.242        -1.967     0.496
C(firmid)[T.24]     -1.6779      0.629     -2.667      0.008        -2.911    -0.445
C(firmid)[T.25]     -0.

## Fixed Effects and clustered standard errors
By combining the previous examples,you can have fixed effects and clustered standard errors at the same time.

In [17]:
firm_cluster_year_fe_ols=sm.ols(formula='y ~ x + C(year)',data=df).fit(
    cov_type='cluster',cov_kwds={'groups':df['firmid']},
    use_t=True)
firm_cluster_year_fe_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     43.23
Date:                Sun, 22 Oct 2017   Prob (F-statistic):           1.93e-61
Time:                        17:07:46   Log-Likelihood:                -10570.
No. Observations:                5000   AIC:                         2.116e+04
Df Residuals:                    4989   BIC:                         2.123e+04
Df Model:                          10                                         
Covariance Type:              cluster                                         
=================================================================================
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept         0.1411      0.089      1.587      0.113        -0.034     0.316
C(year)[T.2]     -0.0119      0.086     -0.139      0.890        -0.181     0.157
C(year)[T.3]     -0.1453      0.086     -1.690      0.092        -0.314     0.024
C(year)[T.4]     -0.2038      0.089     -2.288      0.023        -0.379    -0.029
C(year)[T.5]     -0.0604      0.087     -0.697      0.486        -0.231     0.110
C(year)[T.6]     -0.1312      0.084     -1.562      0.119        -0.296     0.034
C(year)[T.7]     -0.1975      0.087     -2.275      0.023        -0.368    -0.027
C(year)[T.8]     -0.1555      0.094     -1.662      0.097        -0.339     0.028
C(year)[T.9]     -0.1535      0.088     -1.752      0.080        -0.326     0.019
C(year)[T.10]    -0.0556      0.088     -0.634      0.526        -0.228     0.117
x                 1.0351      0.051     20.361      0.000         0.935     1.135
==============================================================================
Omnibus:                        4.804   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.091   Jarque-Bera (JB):                4.752
Skew:                           0.069   Prob(JB):                       0.0929
Kurtosis:                       3.061   Cond. No.                         10.9
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

## Fama-MacBeth Coefficients and standard errors
There is an undocumented Fama-MacBeth function in pandas.However,it is planned to be deprecated and from my (limited) experience,I was unable to replicate standard errors obtained using other packages.Therefore,the most sensible option to me was to write my own function.

In [39]:
def fama_macbeth(formula,time_label,df,lags=3):
    res=df.groupby(time_label).apply(lambda x:sm.ols(
        formula,data=x).fit())
    l=[x.params for x in res]
    p=pd.DataFrame(l)
    
    means={}
    params_labels=res.iloc[0].params.index
    
    #The ':' character used by patsy doesn't play well with pandas
    #column names.
    p.columns=[x.replace(':','_INTER_') for x in p.columns]
    
    for x in p.columns:
        if lags is 0:
            means[x.replace('_INTER_',':')]=sm.ols(formula=x+' ~ 1',
                        data=p[[x]]).fit(use_t=True)
        else:
            means[x.replace('_INTER_',':')]=sm.ols(formula=x+ ' ~ 1',
                        data=p[[x]]).fit(cov_type='HAC',
                                        cov_kwds={'maxlags':lags},
                                        use_t=True)
    params=[]
    stderrs=[]
    tvalues=[]
    pvalues=[]
    for x in params_labels:
        params.append(means[x].params['Intercept'])
        stderrs.append(means[x].bse['Intercept'])
        tvalues.append(means[x].tvalues['Intercept'])
        pvalues.append(means[x].pvalues['Intercept'])
    
    result=pd.DataFrame([params,stderrs,tvalues,pvalues]).T
    result.index=params_labels
    result.columns=['coef','stderr','tvalue','pvalue']
    result['stars']=''
    result.loc[result.pvalue<0.1,'stars']='*'
    result.loc[result.pvalue<0.05,'stars']='**'
    result.loc[result.pvalue<0.01,'stars']='***'
    
    return result
    

In [40]:
fama_macbeth('y ~ x','year',df)

,coef,stderr,tvalue,pvalue,stars
Intercept,0.031278,0.021295,1.468810,1.759490e-01,
x,1.035586,0.025883,40.010988,1.893637e-11,***


It is possible to forgo the Newey-West correction by passing `lags=0`

In [41]:
fama_macbeth('y ~ x','year',df,lags=0)

,coef,stderr,tvalue,pvalue,stars
Intercept,0.031278,0.023356,1.339155,1.805202e-01,
x,1.035586,0.033342,31.059889,8.389156e-212,***


## Newey-West Adjustment for standard errors
The Neway-West adjustment for standard errors is built-in statsmodels with `cov_type=HAC` and the `maxlags` argument passed in the `cov_kwds` parameters.For much details,see https://stackoverflow.com/questions/23420454/newey-west-standard-errors-for-ols-in-python

In [42]:
nw_ols=sm.ols(formula='y ~ x',data=df).fit(cov_type='HAC',
                cov_kwds={'maxlags':3},
                use_t=True)
nw_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     831.8
Date:                Sun, 22 Oct 2017   Prob (F-statistic):          2.48e-169
Time:                        17:54:42   Log-Likelihood:                -10573.
No. Observations:                5000   AIC:                         2.115e+04
Df Residuals:                    4998   BIC:                         2.116e+04
Df Model:                           1                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0297      0.043      0.695      0.487        -0.054     0.113
x              1.0348      0.036     28.841      0.000         0.964     1.105
==============================================================================
Omnibus:                        4.912   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.862
Skew:                           0.070   Prob(JB):                       0.0880
Kurtosis:                       3.063   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 3 lags and without small sample correction
"""

## Discoll-Kraay Standard Errors

In [43]:
dk_ols=sm.ols(formula='y ~ x',data=df).fit(cov_type='nw-groupsum',
                        cov_kwds={'time':df.year,
                                  'groups':df.firmid,
                                  'maxlags':5},
                        use_t=True)
dk_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     2633.
Date:                Sun, 22 Oct 2017   Prob (F-statistic):          3.50e-201
Time:                        17:57:08   Log-Likelihood:                -10573.
No. Observations:                5000   AIC:                         2.115e+04
Df Residuals:                    4998   BIC:                         2.116e+04
Df Model:                           1                                         
Covariance Type:          nw-groupsum                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0297      0.023      1.301      0.194        -0.015     0.075
x              1.0348      0.020     51.317      0.000         0.995     1.074
==============================================================================
Omnibus:                        4.912   Durbin-Watson:                   1.096
Prob(Omnibus):                  0.086   Jarque-Bera (JB):                4.862
Skew:                           0.070   Prob(JB):                       0.0880
Kurtosis:                       3.063   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Driscoll and Kraay Standard Errors are robust to cluster correlation (nw-groupsum)
"""